In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from ngsolve.meshes import MakeStructured2DMesh
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))

In [3]:
R = 1
circle = Circle((0,0),1).Face()
mesh = Mesh(OCCGeometry(circle,dim=2).GenerateMesh(maxh=0.1)).Curve(1+1)

E  = 1e4
nu = 0.3
t  = 1e-2#1e-4#1e-2
q  = 1e-1#1e-8#1e-2
D  = (E*t**3)/(12*(1-nu**2))

In [4]:
def MaterialInv(mat):
    return 1/D*(1/(1-nu)*mat-nu/(1-nu**2)*Trace(mat)*Id(2))

In [5]:
V = HDivDiv(mesh, order=1)
Q = H1(mesh, order=2, dirichlet=".*")
w_1 = NumberSpace(mesh)
fesm = V*Q*w_1

In [6]:
sigma, w, w_1= fesm.TrialFunction()
tau, v, v_1 = fesm.TestFunction()
time = 0.0
dt = 0.5

In [9]:
n = specialcf.normal(2)

def tang(u): return u-(u*n)*n

a = BilinearForm(fesm, symmetric=False)
a += (InnerProduct (MaterialInv(sigma), tau) + div(sigma)*grad(v) \
          + div(tau)*grad(w) )*dx -w_1*v_1*dx\
          + (-(sigma*n) * tang(grad(v)) - (tau*n)*tang(grad(w)))*dx(element_boundary=True)
a.Assemble()

m = BilinearForm(fesm, symmetric=False)
m += w_1*v*dx - w_1*v_1*dx
m.Assemble()

In [10]:
mstar = m.mat.CreateMatrix()
mstar.AsVector().data = m.mat.AsVector() + dt/4 * a.mat.AsVector()
invmstar = mstar.Inverse(freedofs=fesm.FreeDofs())

In [12]:
gfu = GridFunction(fesm)
ft = LinearForm(fesm)
ft += v*dx
time = 0.0
# t.Set(0.0)
gfu.components[2].Set(0.0)


In [14]:
tstep = 10 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
res = gfu.vec.CreateVector()
while t_intermediate < tstep - 0.5 * dt:
#     t.Set(time+t_intermediate)
    ft.Assemble()
    k_1 = invmstar*(ft.vec - a.mat*gfu.vec)
    k_2 = invmstar*(ft.vec-a.mat*(gfu.vec + dt/2*k_1))
    gfu.vec.data = gfu.vec + dt/2*(k_1 +k_2)
    t_intermediate += dt
print("")
time+=t_intermediate